In [ ]:
! pip3 install --user install fuzzywuzzy PyShp geopandas rtree matplotlib cartopy cython cartopy python-Levenshtein

In [2]:
import pandas as pd
import io
import requests
from fuzzywuzzy import process, fuzz
import shapefile
import zipfile
import json
import matplotlib.pyplot as plt
import geopandas as gpd
from cartopy import crs as ccrs
import numpy as np

In [3]:
# read intermediate canonical facilities into dataframe
intermediate_url="https://raw.githubusercontent.com/PTDP/data/main/intermediate_data/geocoded_company_facilities.csv"
intermediate = pd.read_csv(intermediate_url)

hifld_url="https://opendata.arcgis.com/datasets/2d6109d4127d458eaf0958e4c5296b67_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
hifld=pd.read_csv(hifld_url)

In [22]:
# Normalize facility google name

# uppercase
# remove entities before dash IL DOC - BIG MUDDY RIVER CORRECTIONAL CENTER

def clean_facility_internal(str):
    split = str.split("-")
    if (len(split) > 1):
        return split[1]
    return str

intermediate['googlePlaceName'] = intermediate['googlePlaceName'].str.upper();
intermediate['facilityInternal'] = intermediate['facilityInternal'].apply(lambda x: clean_facility_internal(x))

In [6]:
# Output GeoJSON Data
hifld_shp_url="https://opendata.arcgis.com/datasets/2d6109d4127d458eaf0958e4c5296b67_0.zip?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
local_path = 'hifld.zip'

print('Downloading shapefile...')
r = requests.get(hifld_shp_url)

with open(local_path,'wb') as out: ## Open temporary file as bytes
    out.write(io.BytesIO(r.content).read())                ## Read bytes into file

print("Done")

Done


In [48]:
# pre-join > 95 matches in same state
#intermediate['g_name'] = sjoined.apply(lambda x: process.extractOne(x.googlePlaceName, hifld['NAME']), axis=1)
#intermediate['internal_name'] = sjoined.apply(lambda x: process.extractOne(x.facilityInternal, hifld['NAME']), axis=1)
print("hello")

hifld['state'] = hifld['STATE'].str.upper()
hifld['googlePlaceName'] = hifld['NAME']
hifld['facilityInternal'] = hifld['NAME']

intermediate['facilityInternal'] = intermediate['facilityInternal'].str.upper()
intermediate['googlePlaceName'] = intermediate['googlePlaceName'].str.upper()

print(len(intermediate.index))

merged_google = intermediate.merge(hifld, on=['state', 'googlePlaceName'])

merged_facility = intermediate.merge(hifld, on=['state', 'facilityInternal'])

exact_m = pd.concat([merged_google, merged_facility])

# prefer exact matches on facility name
exact_m.drop_duplicates(['uid'], keep='last', inplace=True)

# keep only common columns
exact_removed = pd.concat([intermediate, output], join='inner', ignore_index=True)
exact_removed.drop_duplicates(subset=['uid'], keep=False, inplace=True)

# restore hifld state
hifld.drop(columns=['googlePlaceName', 'facilityInternal', 'state'], inplace=True)

hello
2218


In [50]:
#print(r.content)
# hifld_geojson = shapefile.Reader(local_path + 'Prison_Boundaries.dbf').__geo_interface__
gdf_intermediate = gpd.GeoDataFrame(exact_removed, geometry=gpd.points_from_xy(exact_removed.longitude, exact_removed.latitude))
# remove null
gdf_intermediate = gdf_intermediate[gdf_intermediate.is_valid]
gdf_intermediate.crs = "EPSG:4326"
gdf_intermediate = gdf_intermediate.to_crs("EPSG:3857")

# buffer in miles around each geocoded point
buffer_length_in_meters = (50 * 1000) * 1.60934
gdf_intermediate['geometry'] = gdf_intermediate.geometry.buffer(buffer_length_in_meters)

def fuzz_token (arr):
    print(arr)
    return fuzz.token_set_ratio(arr[0], arr[1])


gdf_hifld = gpd.read_file(local_path)

sjoined = gpd.sjoin(gdf_intermediate, gdf_hifld)

# get similarities for exact match
exact_m["g_name_sim"] = sjoined.apply(lambda x: fuzz.token_set_ratio(x.googlePlaceName, x.NAME), axis=1)
exact_m["internal_name_sim"] = sjoined.apply(lambda x: fuzz.token_set_ratio(x.facilityInternal, x.NAME), axis=1)
exact_m['max_name_similarity'] = sjoined.apply(lambda x: max(x.g_name_sim, x.internal_name_sim), axis=1)
exact_m['hidden'] = False

# get similarities for spatial join
sjoined["g_name_sim"] = sjoined.apply(lambda x: fuzz.token_set_ratio(x.googlePlaceName, x.NAME), axis=1)
sjoined["internal_name_sim"] = sjoined.apply(lambda x: fuzz.token_set_ratio(x.facilityInternal, x.NAME), axis=1)
sjoined['max_name_similarity'] = sjoined.apply(lambda x: max(x.g_name_sim, x.internal_name_sim), axis=1)
sjoined['hidden'] = sjoined.apply(lambda x: x.max_name_similarity < 75 or (pd.isnull(x.FACILITYID)), axis=1)

def make_column(df):
    return pd.Series(np.random.randn(len(df['g_name_sim'])), index=df.index)

sjoined['HIFLDID'] = ""
sjoined['name_override'] =  ""
sjoined['jurisdiction_override'] = ""
sjoined['address_override'] = ""
sjoined['longitude_override'] = ""
sjoined['latitude_overrride'] = ""
sjoined['state_override'] = ""
sjoined['county_override'] = ""
sjoined['countyFIPS_override'] = ""
sjoined['HIFLDID_override'] = ""
sjoined['UCLACovid19ID_override'] = ""
sjoined['hidden_override'] = ""

duplicated = sjoined[sjoined.duplicated('uid')]
duplicated.head()

sjoined_dedupe = sjoined.sort_values('max_name_similarity').drop_duplicates('uid', keep='last')

sjoined_dedupe_m = pd\
    .concat([sjoined_dedupe, gdf_intermediate])\
    .drop_duplicates('uid')\
    .reset_index(drop=True)

output = pd.concat([sjoined_dedupe_m, exact_m])

output['HIFLDID'] = output['FACILITYID']
output.drop(inplace=True, columns=['geometry','index_right','FID','FACILITYID','NAME','ADDRESS','CITY','STATE','ZIP','ZIP4','TELEPHONE','TYPE','STATUS','POPULATION','COUNTY','COUNTYFIPS','COUNTRY','NAICS_CODE','NAICS_DESC','SOURCE','SOURCEDATE','VAL_METHOD','VAL_DATE','WEBSITE','SECURELVL','CAPACITY','SHAPE_Leng','SHAPE_Le_1','SHAPE_Area'])

output.to_csv('joined.csv', encoding="utf-8", index=False)

ValueError: cannot reindex from a duplicate axis